# Генерация "грязных" данных пользователей и покупок

In [11]:
from faker import Faker
import random
import pandas as pd
import numpy as np

fake = Faker()

# "Грязные" данные пользователей
users_dirty = []
professions = ['Engineer', 'Teacher', 'Doctor', 'Artist', 'Developer', 'Manager', 'Scientist', None]
for user_id in range(1, 501):
    # Имя с пропусками и опечатками
    name = fake.name() if random.random() > 0.05 else None
    if name and random.random() < 0.1:
        idx = random.randint(0, len(name)-1)
        name = name[:idx] + random.choice('abcdefghijklmnopqrstuvwxyz') + name[idx+1:]

    # Возраст с аномалиями и пропусками
    r = random.random()
    if r < 0.05:
        age = None
    elif r < 0.1:
        age = random.randint(90, 150)  # аномально большой возраст
    else:
        age = random.randint(18, 70)

    # Город с разными регистрами
    city = fake.city()
    if random.random() < 0.1:
        city = city.lower()

    # Профессия с пропусками и опечатками
    profession = random.choice(professions)
    if profession and random.random() < 0.1:
        profession = profession.lower()
    if profession and random.random() < 0.05:
        idx = random.randint(0, len(profession)-1)
        profession = profession[:idx] + random.choice('abcdefghijklmnopqrstuvwxyz') + profession[idx+1:]

    # Дата присоединения с разными форматами или пропуском
    date_join = fake.date_between(start_date='-5y', end_date='today')
    if random.random() < 0.1:
        date_join = None
    elif random.random() < 0.3:
        date_join = date_join.strftime(random.choice(['%d-%m-%Y', '%Y/%m/%d']))

    users_dirty.append({
        'id': user_id,
        'name': name,
        'age': age,
        'city': city,
        'profession': profession,
        'join_date': date_join
    })


# "Грязные" данные покупок
purchases_dirty = []
categories = ['Electronics', 'Books', 'Clothing', 'Food', 'Toys', 'Sports', 'Cosmetics', None]
for purchase_id in range(1, 3001):
    user_id = random.randint(1, 500)
    purchase_date = fake.date_between(start_date='-2y', end_date='today')
    # Иногда строка с пробелами или None
    r = random.random()
    if r < 0.05:
        amount = None
    elif r < 0.15:
        amount_val = round(random.uniform(10, 1000), 2)
        amount = str(amount_val) + " "
        if random.random() < 0.3:
            amount = amount.replace('.', ',')
    else:
        amount = round(random.uniform(10, 1000), 2)

    # Категория с пропусками и разным регистром
    category = random.choice(categories)
    if category and random.random() < 0.2:
        category = category.lower()
    if random.random() < 0.05:
        category = None

    # Дата покупки как объект или строка в разных форматах
    date_r = random.random()
    if date_r < 0.1:
        purchase_date = None
    elif date_r < 0.4:
        purchase_date = purchase_date.strftime(random.choice(['%m-%d-%Y', '%d/%m/%Y']))

    purchases_dirty.append({
        'id': purchase_id,
        'user_id': user_id,
        'purchase_date': purchase_date,
        'amount': amount,
        'category': category
    })

# Преобразуем в DataFrame
df_users_dirty = pd.DataFrame(users_dirty)
df_purchases_dirty = pd.DataFrame(purchases_dirty)

# сохраняем наши датафреймы в файл excel на разных листах
with pd.ExcelWriter('marketplace_users_data.xlsx') as writer:
    df_users_dirty.to_excel(writer, sheet_name='users', index=False)
    df_purchases_dirty.to_excel(writer, sheet_name='purchases', index=False)
